# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [48]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [49]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

We'll make a first selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [61]:
# Load necessary packages
import numpy as np
from sklearn.impute import SimpleImputer

# remove "object"-type features and SalesPrice from `X`
X = df.drop(df.select_dtypes(['object']), axis=1)
X = X[[x for x in X.columns if x != 'SalePrice']]

# Impute null values
impute = SimpleImputer(strategy='median')
X = pd.DataFrame(impute.fit_transform(X), columns=X.columns)

# Create y
y = df.SalePrice

Look at the information of `X` again

In [62]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null float64
MSSubClass       1460 non-null float64
LotFrontage      1460 non-null float64
LotArea          1460 non-null float64
OverallQual      1460 non-null float64
OverallCond      1460 non-null float64
YearBuilt        1460 non-null float64
YearRemodAdd     1460 non-null float64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null float64
BsmtFinSF2       1460 non-null float64
BsmtUnfSF        1460 non-null float64
TotalBsmtSF      1460 non-null float64
1stFlrSF         1460 non-null float64
2ndFlrSF         1460 non-null float64
LowQualFinSF     1460 non-null float64
GrLivArea        1460 non-null float64
BsmtFullBath     1460 non-null float64
BsmtHalfBath     1460 non-null float64
FullBath         1460 non-null float64
HalfBath         1460 non-null float64
BedroomAbvGr     1460 non-null float64
KitchenAbvGr     1460 non-null floa

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [63]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Split in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

# Fit the model and print R2 and MSE for train and test
lin = LinearRegression()
lin.fit(X_train, y_train)
y_hat = lin.predict(X_test)

print("R2 Score:", r2_score(y_hat, y_test))
print("MSE: ", mean_squared_error(y_hat, y_test))

R2 Score: 0.8044562150546132
MSE:  890831684.4746418


## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [64]:
from sklearn import preprocessing

# Scale the data and perform train test split
X_scale = pd.DataFrame(preprocessing.scale(X), columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=0.2, random_state=4)

Perform the same linear regression on this data and print out R-squared and MSE.

In [65]:
# Your code here
lin = LinearRegression()
lin.fit(X_train, y_train)
y_hat = lin.predict(X_test)

print("R2 Score:", r2_score(y_hat, y_test))
print("MSE: ", mean_squared_error(y_hat, y_test))

R2 Score: 0.8045079878157659
MSE:  889805531.7659816


## Include dummy variables

We haven't included dummy variables so far: let's use our "object" variables again and create dummies

In [66]:
# Create X_cat which contains only the categorical variables
X_cat =  df.select_dtypes(['object'])


In [67]:
# Make dummies
X_cat = pd.get_dummies(X_cat)

Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [69]:
# Your code here
X = pd.concat([X_scale, X_cat], axis=1)
X.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-1.730865,0.073375,-0.220875,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,0.575425,...,0,0,0,1,0,0,0,0,1,0
1,-1.728492,-0.872563,0.460320,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,1.171992,...,0,0,0,1,0,0,0,0,1,0
2,-1.726120,0.073375,-0.084636,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,0.092907,...,0,0,0,1,0,0,0,0,1,0
3,-1.723747,0.309859,-0.447940,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.499274,...,0,0,0,1,1,0,0,0,0,0
4,-1.721374,0.073375,0.641972,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,0.463568,...,0,0,0,1,0,0,0,0,1,0


Perform the same linear regression on this data and print out R-squared and MSE.

In [70]:
# Your code here
# Split in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

# Fit the model and print R2 and MSE for train and test
lin = LinearRegression()
lin.fit(X_train, y_train)
y_hat = lin.predict(X_test)

print("R2 Score:", r2_score(y_hat, y_test))
print("MSE: ", mean_squared_error(y_hat, y_test))

R2 Score: -0.004199038251905263
MSE:  2.61028642009436e+30


Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far far off. Similarly, the scale of the Testing MSE is orders of magnitude higher then that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [71]:
# Your code here
lasso = Lasso(alpha=1)
lasso.fit(X_train, y_train)
y_hat = lasso.predict(X_test)

print("R2 Score:", r2_score(y_hat, y_test))
print("MSE: ", mean_squared_error(y_hat, y_test))

R2 Score: 0.8841716965858458
MSE:  680394962.4573069


With a higher regularization parameter (alpha = 10)

In [72]:
# Your code here
# Your code here
lasso = Lasso(alpha=10)
lasso.fit(X_train, y_train)
y_hat = lasso.predict(X_test)

print("R2 Score:", r2_score(y_hat, y_test))
print("MSE: ", mean_squared_error(y_hat, y_test))

R2 Score: 0.8911004013761705
MSE:  616636975.8938768


## Ridge

With default parameter (alpha = 1)

In [77]:
# Your code here
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)
y_hat = ridge.predict(X_test)

print("R2 Score:", r2_score(y_hat, y_test))
print("MSE: ", mean_squared_error(y_hat, y_test))

R2 Score: 0.8778105234879876
MSE:  672700497.5243238


With default parameter (alpha = 10)

In [78]:
# Your code here
# Your code here
ridge = Ridge(alpha=10)
ridge.fit(X_train, y_train)
y_hat = ridge.predict(X_test)

print("R2 Score:", r2_score(y_hat, y_test))
print("MSE: ", mean_squared_error(y_hat, y_test))

R2 Score: 0.8788560326388636
MSE:  624969026.8086439


## Look at the metrics, what are your main conclusions?

Lasso regression seemed to perform slightly better than ridge regression. This is most likely since Lasso drops more coefficients and our parameter space is already quite high.

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

In [80]:
# number of Ridge params almost zero
(np.abs(ridge.coef_) <= 0.001).sum()

3

In [81]:
# number of Lasso params almost zero
(np.abs(lasso.coef_) <= 0.001).sum()

73

Compare with the total length of the parameter space and draw conclusions!

In [82]:
# your code here
len(lasso.coef_)

289

## Summary

Great! You now know how to perform Lasso and Ridge regression.